In [1]:
import phrase_entity_extraction as ph

In [11]:
sent = "111 CENTER PARK DRIVE STE 210".lower()

def return_best_fit(seq, sent):
    def get_sorted_entity(_markers, entity):
        entities = [arr for arr in _markers if arr[3][0] == entity]
        entities.sort(key=lambda x: int(x[2]))
        return entities
    def entitys_overlap(ent1, ent2):
        if ent1[1] <= ent2[0] or ent2[1] <= ent1[0]:
            return False
        else:
            return True
    def add_next(markers, best_fit, entity):
        suites = get_sorted_entity(markers, entity)
        print("sweets: ", suites, entity)
        idx = len(suites) - 1
#         if not best_fit and idx == 0:
#             best_fit.append(suites[idx])
#             return best_fit
        while idx >= 0:
            print("we are on")

            if not any([entitys_overlap(item, suites[idx]) for item in best_fit]):
                best_fit.append(suites[idx])
                break
            else:
                idx -= 1
        return best_fit
    markers = ph.get_markers(ph.seq, sent, ['ADDRESS', 'POBOX', 'SUITE', 'ATTN','_DIR_'])
    best_fit = []
    for nugget in ['POBOX', 'ADDRESS', 'ATTN', 'SUITE','_DIR_']:
        print("best_fit: ", best_fit)
        best_fit = add_next(markers, best_fit, nugget)

    best_fit.sort(key=lambda x: int(x[0]))
    return best_fit

result = return_best_fit(ph.seq, sent)
print(result)
def return_max_address3(seq, sent):
    results = return_best_fit(seq,sent)
    addresses = []
    for result in results:
        if result[3][0] in ['ADDRESS', 'SUITE', '_DIR_']:
            addresses.append(result[4])
    if not addresses:
        if result[3][0] in ['POBOX']:
            addresses.append(result[4])
    return " ".join(addresses)
print("HERE: ",return_max_address3(ph.seq, sent))

best_fit:  []
sweets:  [] POBOX
best_fit:  []
sweets:  [[0, 3, 3, ['ADDRESS'], '111 center park']] ADDRESS
we are on
best_fit:  [[0, 3, 3, ['ADDRESS'], '111 center park']]
sweets:  [] ATTN
best_fit:  [[0, 3, 3, ['ADDRESS'], '111 center park']]
sweets:  [[0, 1, 1, ['SUITE'], '111'], [5, 6, 1, ['SUITE'], '210'], [4, 6, 2, ['SUITE'], 'ste 210']] SUITE
we are on
best_fit:  [[0, 3, 3, ['ADDRESS'], '111 center park'], [4, 6, 2, ['SUITE'], 'ste 210']]
sweets:  [] _DIR_
[[0, 3, 3, ['ADDRESS'], '111 center park'], [4, 6, 2, ['SUITE'], 'ste 210']]
best_fit:  []
sweets:  [] POBOX
best_fit:  []
sweets:  [[0, 3, 3, ['ADDRESS'], '111 center park']] ADDRESS
we are on
best_fit:  [[0, 3, 3, ['ADDRESS'], '111 center park']]
sweets:  [] ATTN
best_fit:  [[0, 3, 3, ['ADDRESS'], '111 center park']]
sweets:  [[0, 1, 1, ['SUITE'], '111'], [5, 6, 1, ['SUITE'], '210'], [4, 6, 2, ['SUITE'], 'ste 210']] SUITE
we are on
best_fit:  [[0, 3, 3, ['ADDRESS'], '111 center park'], [4, 6, 2, ['SUITE'], 'ste 210']]
sweets:

In [12]:
ph.get_markers(ph.seq, sent, ['ADDRESS', 'SUITE','POBOX', 'ATTN','_DIR_'])

[[0, 1, 1, ['SUITE'], '111'],
 [0, 3, 3, ['ADDRESS'], '111 center park'],
 [4, 6, 2, ['SUITE'], 'ste 210'],
 [5, 6, 1, ['SUITE'], '210']]

In [4]:
def get_interpretation(seq, arr_words, arr_search_keys):
    """INPUT: ['123', 'main']"""
    t1_encoded = ph.encode_from_word_list(arr_words)
    #print("encoded: " ,t1_encoded)
    seq.look_ahead(t1_encoded)
    next_values = seq.get_next_values()
    #print("returning ", next_values)
    return [search_key for search_key in arr_search_keys if search_key in next_values]

#get_interpretation(seq, w("123 martin luther king jr"))
get_interpretation(ph.seq, ph.w(sent), ['ADDRESS', 'POBOX', 'SUITE','DIR'])

[]

In [5]:
def index_tail(lst):
    """Return the 0 based index of last element in lst"""
    return len(lst) - 1
def validate_back_jump(last_length, idx):
    """Validate jumping back to start of sequence, check if any other back sequence is in the way"""
    current_range = last_length[idx]
    while current_range > 1:
        print("index: ", idx, " range: ", current_range, " last ", last_length[idx-1], " <? ", current_range -1)
        if last_length[idx-1] <= current_range - 1:
            current_range -= 1
            idx -= 1
            continue
        else:
            return False
    return True


def decompose_into_dictionary_words(domain, _seq, types):
    last_length = [-1] * len(domain)
    last_interp = ['']*len(domain)
    #print("Length Domain: ", len(domain))
    for i in range(len(domain)):
        #print("INPUT TO GET_INTERPRETATION: ", domain[:i + 1])
        inter1 = get_interpretation(_seq, domain[:i + 1], types)
        if any([ret_type in types for ret_type in inter1]):
            #print("HIT A")
            last_interp[i] = [hit for hit in inter1 if hit in types]
            last_length[i] = i + 1
            
        if last_length[i] == -1:
            for j in range(i):
                inter2 = get_interpretation(_seq, domain[j + 1:i + 1], types)
                #print("SUB INPUT for INTERPRET: ", domain[j+1:i+1], inter2, last_length[j])
                #if last_length[j] != -1 and any([ret_type in types for ret_type in inter2]):
                if any([ret_type in types for ret_type in inter2]):
                    #print('HIT B')
                    last_interp[i] = [hit for hit in inter2 if hit in types]
                    last_length[i] = i - j
                    break
        #print("last_length: ", last_length)
        #print("last_interp: ", last_interp)
    #return last_length, last_interp
    print("last_length: ", last_length)
    print("last_interp: ", last_interp)
    #print("BEGIN DECOMPOSITON PHASE")
    decompositions = []
    components = []

    idx = index_tail(domain)
    while idx >= 0:
        #print("idx: ", idx)
        if validate_back_jump(last_length, idx):
            decompositions.append(last_interp[idx])
            components.append((last_interp[idx], " ".join(domain[idx + 1 - last_length[idx]:idx + 1])))
        if last_length[idx] == -1:
            idx -= 1
            continue
        if validate_back_jump(last_length, idx):
            idx -= last_length[idx]
        else:
            idx -= 1
    decompositions = decompositions[::-1]
    components = components[::-1]
    print("Decompositions: ", decompositions)
    return last_length, last_interp, decompositions, components

decompose_into_dictionary_words(ph.w(sent.lower()), ph.seq, ['ADDRESS', 'POBOX', 'SUITE', 'ATTN'])

last_length:  [1, -1, 3, 4, -1]
last_interp:  [['SUITE'], '', ['ADDRESS'], ['ADDRESS'], '']
index:  3  range:  4  last  3  <?  3
index:  2  range:  3  last  -1  <?  2
index:  1  range:  2  last  1  <?  1
index:  3  range:  4  last  3  <?  3
index:  2  range:  3  last  -1  <?  2
index:  1  range:  2  last  1  <?  1
Decompositions:  [['ADDRESS'], '']


([1, -1, 3, 4, -1],
 [['SUITE'], '', ['ADDRESS'], ['ADDRESS'], ''],
 [['ADDRESS'], ''],
 [(['ADDRESS'], '1383 village park dr'), ('', '')])

In [6]:
def return_max_address2(seq, sent):
    kinds = ['ADDRESS', 'POBOX', 'SUITE']
    decomposition = decompose_into_dictionary_words(ph.w(sent.lower()), seq, kinds)
    found_tuples = decomposition[3]
    max_address = []
    for kindof, value in found_tuples:
        if kindof and kindof[0] in kinds:
            max_address.append(value)
    return " ".join(max_address).upper()
            
return_max_address2(ph.seq, sent)

last_length:  [1, -1, 3, 4, -1]
last_interp:  [['SUITE'], '', ['ADDRESS'], ['ADDRESS'], '']
index:  3  range:  4  last  3  <?  3
index:  2  range:  3  last  -1  <?  2
index:  1  range:  2  last  1  <?  1
index:  3  range:  4  last  3  <?  3
index:  2  range:  3  last  -1  <?  2
index:  1  range:  2  last  1  <?  1
Decompositions:  [['ADDRESS'], '']


'1383 VILLAGE PARK DR'

In [7]:
ph.suite_sequences

[[['ALNUM']],
 [['DIGIT']],
 [['DIGIT'], ['DIR']],
 [['APT'], ['DIGIT'], ['DIR']],
 [['NUMS_1AL']],
 [['DIGDASH']],
 [['LETTER']],
 [['APT_NUM']],
 [['POUNDDIG']],
 [['NTH'], ['APT']],
 [['APT'], ['ALPHA']],
 [['APT'], ['ALNUM']],
 [['APT'], ['DIGIT']],
 [['APT'], ['DIGIT'], ['APT'], ['ALNUM']],
 [['APT'], ['DIGIT'], ['DIGIT']],
 [['APT'], ['LETTER'], ['DIGIT']],
 [['APT'], ['LETTER'], ['APT'], ['DIGIT']],
 [['APT'], ['POUND'], ['DIGIT']],
 [['APT'], ['DIGIT'], ['APT'], ['DIGIT']],
 [['APT'], ['DIGDASH']],
 [['APT'], ['DIGDASHAL']],
 [['APT'], ['LETTER']],
 [['APT'], ['NUMS_1AL']],
 [['POUND'], ['ALPHA']],
 [['POUND'], ['ALNUM']],
 [['POUND'], ['DIGIT']],
 [['DIGIT'], ['DIR'], ['POUND'], ['DIGIT']],
 [['POUND'], ['DIGDASH']],
 [['POUND'], ['DIGDASHAL']],
 [['POUND'], ['LETTER']],
 [['POUND'], ['NUMS_1AL']]]